<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Example_Implementing_OAuth2_Authentication_in_Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, redirect, url_for
from flask_oauthlib.client import OAuth

app = Flask(__name__)
oauth = OAuth(app)

google = oauth.remote_app(
    'google',
    consumer_key='YOUR_GOOGLE_CLIENT_ID',
    consumer_secret='YOUR_GOOGLE_CLIENT_SECRET',
    request_token_params={
        'scope': 'email',
    },
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth',
)

@app.route('/')
def index():
    return 'Welcome to the OAuth2 example!'

@app.route('/login')
def login():
    return google.authorize(callback=url_for('authorized', _external=True))

@app.route('/logout')
def logout():
    return 'You have been logged out.'

@app.route('/login/authorized')
def authorized():
    response = google.authorized_response()

    if response is None or 'access_token' not in response:
        return 'Access denied: reason={} error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )

    access_token = response['access_token']

    user_info = google.get('userinfo')

    return 'Logged in as: ' + user_info.data['email']

if __name__ == '__main__':
    app.run(debug=True)